## Import Libs

In [ ]:
# import os
import pandas as pd
import numpy as np
import datetime as dt
import logging
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

from torch import optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from torch.optim import AdamW
from utils import SklearnWrapper

In [ ]:
from config import *
from entities import *
from components import *
from strategies import *
from datasets import *
from engine import Engine
from models import DiffusionTransformer
from frameworks import Diffusion

## Setting 

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

## Data

### Load Data to basket

In [ ]:
symbols = ['AAPL', 'TSLA', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'GOOG', 'META', 'AVGO', 'ORCL', 'CRM', 'ADBE', 'AMD', 'CSCO']
basket = Basket(symbols=symbols)
basket.load_all_assets(freq="1d")

### Norm To Returns

In [ ]:
targets = ["Close"]
for symbol, asset in basket.assets.items():
    asset.to_returns(log=True, columns=targets)
asset.data.head(5)

### Align Data (Joint Data)

In [ ]:
strategy = IntersectionStrategy()
joint_df = basket.align(strategy)
joint_df.head()

In [ ]:
features = ["Close (Log_Returns)"]
basket_tensor = basket.to_tensor(features=features)
basket_tensor.shape

In [ ]:
def verify_scaling(t: torch.Tensor):
    print(f"Max: {t.max().item():.4f}")
    print(f"Min: {t.min().item():.4f}")
    print(f"Mean: {t.mean().item():.4f}")
    print(f"Std: {t.std().item():.4f}")

verify_scaling(basket_tensor)

#### Market Setup & Window Rolling Setup

In [ ]:
window = RollingWindow(size=64, stride=1)
market = Market(basket, window)
market_tensor = market.setup(features=features)

In [ ]:
print(f"Market Tensor Shape: {market_tensor.shape}")
verify_scaling(market_tensor)

### Scaler

In [ ]:
sklearn_scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = SklearnWrapper(sklearn_scaler)

scaler.fit(market_tensor)
norm_data = scaler.encode(market_tensor)

print(f"Norm data shape: {market_tensor.shape}")
print(f"Data {market_tensor[0,0,0,:]}")
verify_scaling(norm_data)

### Dataloader & Datasets

In [ ]:
modes = ["exhaustive", "random"]
market_ds = JointMarketDataset(norm_data)

print(f"Norm Data: {norm_data.shape}")
print(f"Market Dataset: {len(market_ds)}")

# market_ds = MarketDataset(norm_data, mode='random')
ratios = [0.8, 0.1, 0.1]
train_ds, val_ds, test_ds = create_randomize_datasets(market_ds, ratios)

print(f"Train samples: {len(train_ds)}")
print(f"Val samples: {len(val_ds)}") 
print(f"Test samples: {len(test_ds)}")

In [ ]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=False)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=True)

In [ ]:
x = next(iter(train_loader))
print(f"x shape: {x.shape}")
verify_scaling(x)

## Model

### Setup

In [ ]:
cfg = TrainConfig(
    epochs=100,
    optimizer=OptimizerConfig(lr=2e-4)
)

print(cfg.optimizer.lr) 
print(cfg.epochs)    
print(cfg.ddpm)

In [ ]:
WINDOW_SIZE = x.shape[1]       # T
NUM_ASSETS = x.shape[2]         # N
NUM_FEATURES = x.shape[3]      # F
TOTAL_INPUT_DIM = NUM_ASSETS * NUM_FEATURES # C = N * F (Flatten)

print(f"Running on: {cfg.device}")
print(f"Input Dimension (Channels): {TOTAL_INPUT_DIM}")

### Diffusion

In [ ]:
diffusion = Diffusion(
    noise_steps=cfg.ddpm.noise_steps,
    beta_start=cfg.ddpm.beta_start,
    beta_end=cfg.ddpm.beta_end,
    schedule=cfg.scheduler.type,
    device=cfg.device
)

### NN Model

In [ ]:
model = DiffusionTransformer(
    features_in=TOTAL_INPUT_DIM,  # รับ Input ขนาด N*F
    d_model=cfg.ddpm.d_model,                  # ความกว้าง Model
    nhead=cfg.ddpm.n_heads,
    num_layers=cfg.ddpm.n_layers,
    max_len=WINDOW_SIZE           # รองรับความยาวสูงสุดเท่า Window
).to(cfg.device)

### Optimizer

In [ ]:
optimizer = AdamW(model.parameters(), lr=cfg.optimizer.lr, weight_decay=cfg.optimizer.weight_decay)

## Engine

In [ ]:
engine = Engine(
    model=model,
    diffusion=diffusion,
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    optimizer=optimizer,
    device=cfg.device,
    scaler=scaler
)

### Training

In [ ]:
engine.fit(epochs=cfg.epochs, save_dir="./checkpoints")

### Test

In [ ]:
x_test = next(iter(test_loader))
x_test.shape

In [ ]:
verify_scaling(x_test)

In [ ]:
prediction = engine.simulate(x_test, steps_to_predict=8)
prediction.shape

In [ ]:
verify_scaling(prediction)

## Monte Carlo Sim (GenAI)

In [ ]:
@torch.no_grad()
def monte_carlo_simulate(context_data, steps_to_simulate, engine,num_simulations=100):
    x_context = context_data.repeat(num_simulations, 1, 1, 1).to(engine.device)
    mc_results = engine.simulate(x_context, steps_to_simulate)
    return mc_results

In [ ]:
x_test_sampling = x_test[0].unsqueeze(0)
x_test_sampling.shape

In [ ]:
mc_y = monte_carlo_simulate(context_data=x_test_sampling, steps_to_simulate=24, engine=engine,num_simulations=100)
mc_y.shape # [Sims, Time, Assets, Features]

## Monete Carlo Statistic

In [ ]:
def gbm_monte_carlo_simulate(hist_log_returns: np.ndarray, steps_to_predict: int, num_sims: int = 100, dt: float = 1.0):
    """ Arg: 
            dt: Time increment
        Returns: 
            future_prices: [num_sims, steps_to_predict]
    """
    mu = np.mean(hist_log_returns)
    sigma = np.std(hist_log_returns)

    # Random Shock (Z) from Normal Dist (0, 1) -> shape: [num_sims, steps_to_predict]
    Z = np.random.normal(0, 1, (num_sims, steps_to_predict))

    # Future log returns (GBM)
    # r_t = (mu - 0.5 * sigma^2) * dt + sigma * sqrt(dt) * Z
    drift_term = (mu - 0.5 * sigma**2) * dt
    shock_term = sigma * np.sqrt(dt) * Z

    future_daily_log_returns = drift_term + shock_term
    cum_log_returns = np.cumsum(future_daily_log_returns, axis=1)

    # future_prices = last_price * np.exp(cum_log_returns)
    
    return future_daily_log_returns

In [ ]:
x_test_sampling.shape

In [ ]:
x_test_statistic_sampling = x_test_sampling[0,:,:,:]
x_test_statistic_sampling.shape

In [ ]:
x_test_statistic_sampling = x_test_statistic_sampling[:,0,0]
x_test_statistic_sampling.shape

In [ ]:
type(x_test_statistic_sampling)

In [ ]:
mc_paths = gbm_monte_carlo_simulate(
    hist_log_returns=x_test_statistic_sampling.cpu().numpy(), 
    steps_to_predict=14, 
    num_sims=1000
)
type(mc_paths)

In [ ]:
asset_tests = x_test_sampling[0, :,:, :]
asset_tests.shape

In [ ]:
asset_tests = asset_tests.permute(1,0,2)
asset_tests.shape

In [ ]:
mc_simed_ret_assets = []
for asset in asset_tests:
    mc_sim_ret = gbm_monte_carlo_simulate(
        hist_log_returns=asset[:, 0].cpu().numpy(), 
        steps_to_predict=14, 
        num_sims=1000
    )
    mc_simed_ret_assets.append(mc_sim_ret)

In [ ]:
len(mc_simed_ret_assets)

In [ ]:
for asset in mc_simed_ret_assets:
    mc_ret_context = asset[:, -1]
    mc_ret_sim = asset[-1, :]

    mean_ret_sim = np.mean(mc_ret_sim)
    std_ret_sim = np.std(mc_ret_sim)
    upper_bound = np.percentile(mc_ret_sim, 95) # Best case 5%
    lower_bound = np.percentile(mc_ret_sim, 5) # Worst case 5%
    print(f"mean: {mean_ret_sim}, std: {std_ret_sim}")
    print(f"lower bound: {lower_bound}, upper bound:{upper_bound}")

In [ ]:
sim_results_per_asset = []

for asset_sim in mc_simed_ret_assets:
    # asset_sim shape: (1000, 14) -> Daily Log Returns
    
    # รวม Log Return 14 วัน เป็นก้อนเดียว (Total Log Return)
    # axis=1 คือรวมตามแกนเวลา (Time)
    total_return = np.sum(asset_sim, axis=1) # ได้ shape (1000,)
    
    sim_results_per_asset.append(total_return)

# 2. แปลงเป็น Matrix (Num_Assets, Num_Sims)
# เช่น (3, 1000)
comparison_matrix = np.array(sim_results_per_asset)

# 3. คำนวณ Correlation Matrix
corr_matrix = np.corrcoef(comparison_matrix)

print("Correlation Matrix:")
print(corr_matrix)

# (แถม) วาด Heatmap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

## Experiment

### Ground Truth Stats

In [ ]:
x_test = next(iter(test_loader))
x_test_sampling = x_test[2, :, :, :]
x_test_sampling.shape

In [ ]:
x_test_sampling_np = x_test_sampling.cpu().numpy()
print(type(x_test_sampling))
print(type(x_test_sampling_np))

In [ ]:
def calc_stats(x):
    if x.ndim == 2:
        # Shape: [64, 3] -> [L, N]
        print(x.shape)
        mean = np.mean(x, axis=0)
        std = np.std(x, axis=0)
        avg_corr = np.corrcoef(x.T)

    # Calc All na
    if x.ndim == 3:
        # Shape: (1000, 64, 3) -> [Num of sims, L, N]
        mean = np.mean(x, axis=(0, 1))
        std = np.std(x, axis=(0, 1))

        corrs = []
        for i in range(x.shape[0]):
            c_mat = np.corrcoef(x[i].T)
            corrs.append(c_mat)

        avg_corr = np.mean(corrs,axis=0)
    return mean, std, avg_corr

In [ ]:
gt_mean, gt_std, gt_corr = calc_stats(x_test_sampling_np[:,:,-1])

In [ ]:
gt_corr

In [ ]:
print("--- Ground Truth Stats ---")
print(f"Mean: {gt_mean}")
print(f"Std:  {gt_std}")
print(f"Corr Matrix:\n{gt_corr}")
print("-" * 30)

### MC Stats

In [ ]:
length , num_assets, num_features = x_test_sampling.shape

# Config
SPLIT_IDX = 60
STEPS_SIMS = 4
NUM_SIMS = 1000


In [ ]:
x_test_sampling.shape

In [ ]:
y = x_test_sampling[-4:]
y.shape

In [ ]:
context_data = x_test_sampling[:SPLIT_IDX, :]
context_data.shape

In [ ]:
mc_sims_paths = []
for i in range(num_assets):
    print("---" * 30)
    asset_context = context_data[:, i][:, 0]
    print(f"Shape: {asset_context.shape}")
    mc_pred_returns = gbm_monte_carlo_simulate(
        hist_log_returns=asset_context.cpu().numpy(), 
        steps_to_predict=STEPS_SIMS, 
        num_sims=NUM_SIMS
    )
    print(f"MC Pred Returns Shape: {mc_pred_returns.shape}")
    
    context_expanded = np.tile(asset_context.cpu(), (NUM_SIMS, 1))
    print(f"Context Expanded Shape: {context_expanded.shape}")
    
    full_paths = np.hstack([context_expanded, mc_pred_returns])
    print(f"Full Paths Shape: {full_paths.shape}")
    mc_sims_paths.append(full_paths)

#### Inside detail in the loop

In [ ]:
mc_pred_returns = gbm_monte_carlo_simulate(
    hist_log_returns=asset_context.cpu().numpy(), 
    steps_to_predict=STEPS_SIMS, 
    num_sims=NUM_SIMS
)
mc_pred_returns.shape

In [ ]:
context_expanded = np.tile(asset_context.cpu(), (NUM_SIMS, 1))
context_expanded.shape

In [ ]:
full_paths = np.hstack([context_expanded, mc_pred_returns])
full_paths.shape

#### MC Stats Calc

In [ ]:
mc_sims_paths[0].shape

In [ ]:
mc_sims = np.stack(mc_sims_paths, axis=-1)
mc_sims.shape

In [ ]:
type(mc_sims)

In [ ]:
mc_sims_mean, mc_sims_std, mc_sims_corr = calc_stats(mc_sims)

In [ ]:
print("--- MC Stats ---")
print(f"Mean: {mc_sims_mean}")
print(f"Std:  {mc_sims_std}")
print(f"Corr Matrix:\n{mc_sims_corr}")
print("-" * 30)

## MC (GenAI) Stats

In [ ]:
# x_test_sampling = x_test[0].unsqueeze(0)
x_test_sampling.shape

In [ ]:
length , num_assets, num_features = x_test_sampling.shape

# Config
SPLIT_IDX = 60
STEPS_SIMS = 4
NUM_SIMS = 1000


In [ ]:
mc_genai_y = x_test_sampling[-4:]
mc_genai_y.shape

In [ ]:
mc_genai_context_x = x_test_sampling[:SPLIT_IDX, :]
mc_genai_context_x.shape

In [ ]:
mc_genai_context_x = mc_genai_context_x.unsqueeze(0)
mc_genai_context_x.shape

In [ ]:
# Shape requires [1, 64, 3, 2]
mc_y = monte_carlo_simulate(context_data=mc_genai_context_x, steps_to_simulate=STEPS_SIMS, engine=engine,num_simulations=NUM_SIMS)
mc_y.shape # [Sims, Time, Assets, Features]

In [ ]:
# Shape require for calc stats is (1000, 64, 3)
mc_genai_y = mc_y[:,:,:,0]
mc_genai_y.shape

In [ ]:
mc_genai_mean, mc_genai_std, mc_genai_corr = calc_stats(mc_genai_y)
print("--- MC GenAI Stats ---")
print(f"Mean: {mc_genai_mean}")
print(f"Std:  {mc_genai_std}")
print(f"Corr Matrix:\n{mc_genai_corr}")
print("-" * 30)

In [ ]:
mc_genai_corr, mc_sims_corr, gt_corr


In [ ]:
sns.heatmap(mc_genai_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

## Optimize Portfolio

In [ ]:
genai_y = mc_genai_y[0, :, :]

print(f"MC GenAI Shape: {mc_genai_y.shape}")
print(f"GenAI Samping Shape: {genai_y.shape}")

In [ ]:
vols_genai = np.std(mc_genai_y, axis=1)

print(f"GenAI Volatilities Shape: {vols_genai.shape}")
print(f"Sample GenAI Vol: {vols_genai[0]}")

In [ ]:
cov_matrix_genai = np.array([np.cov(sim, rowvar=False) for sim in mc_genai_y])

print(f"GenAI Covariance Matrices Shape: {cov_matrix_genai.shape}")
print(f"GenAI Covariance Matrix: {cov_matrix_genai[0].shape}")

In [ ]:
vols_outer_genai = vols_genai[:, :, None] * vols_genai[:, None, :]

print(f"GenAI Vols Outer Shape: {vols_outer_genai.shape}")

corr_matrix_genai = cov_matrix_genai / vols_outer_genai
print(f"GenAI Corr Matrix Shape: {corr_matrix_genai.shape}")
print(f"GenAI Corr Matrix: {corr_matrix_genai[0][0]}")

In [ ]:
sns.heatmap(corr_matrix_genai[0], annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

In [ ]:
expected_returns_genai = mc_genai_y.mean(axis=1)
print(f"GenAI Expected Returns Shape: {expected_returns_genai.shape}")

mean_expected_returns_genai = expected_returns_genai.mean(axis=0)
print(f"GenAI Mean of Expected Returns Shape: {mean_expected_returns_genai.shape}")
print(f"GenAI Mean of Expected Returns: {mean_expected_returns_genai}")

In [ ]:
def calc_sharp_ratio(expected_returns, risk_free_rate, std):
    return (expected_returns - risk_free_rate) / std

print(vols_genai.shape, expected_returns_genai.shape)

risk_free_rate = .02
risk_free_rates = np.full(vols_genai.shape, risk_free_rate)

print(risk_free_rates.shape)
sharp_ratio = calc_sharp_ratio(expected_returns=expected_returns_genai, risk_free_rate=risk_free_rates, std=vols_genai)
print(sharp_ratio.shape)
print(sharp_ratio[0][0])
# for exp_r_genai in expected_returns_genai:
    # sharp_ratio = calc_sharp_ratio()

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights) 
    
    # std Port = sqrt(w.T * Cov * w)
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    return returns, std_dev

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.02):
    p_ret, p_var = portfolio_performance(weights, mean_returns, cov_matrix)
    
    # Sharpe Ratio = (Return - RiskFree) / Volatility
    return -(p_ret - risk_free_rate) / p_var

In [ ]:
from scipy.optimize import minimize

sim, t, num_assets = mc_genai_y.shape
args = (expected_returns_genai[0], cov_matrix_genai[0] )

# Constraints: sum of weights must equals 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# Bounds: weight 0 to 1 no (short sell)
bounds = tuple((0.0, 1.0) for asset in range(num_assets))
# Initial Guess: Start at equal weight ratio
init_guess = num_assets * [1. / num_assets,]

result = minimize(negative_sharpe_ratio, init_guess, args=args,
                  method='SLSQP', bounds=bounds, constraints=constraints)

1. **Simple Volatility**: Rolling Simple Volatity defined as
$$
\hat{\sigma}_t = \sqrt{\frac{1}{N-1} \sum_{i=1}^N (r_{t-i} - \bar{r})^2}
$$

In [ ]:
if result.success:
    optimized_weights = result.x
    print(f"Optimized Weight: {optimized_weights}")
    opt_return, opt_vol = portfolio_performance(optimized_weights, expected_returns_genai[0], cov_matrix_genai[0])
    opt_sharpe = -result.fun
    print("Optimization Successful!")
    print("-" * 30)
    print(f"Optimal Weights: {np.round(optimized_weights, 4)}")
    print(f"Expected Return: {opt_return:.4f} ({opt_return*100:.2f}%)")
    print(f"Volatility     : {opt_vol:.4f} ({opt_vol*100:.2f}%)")
    print(f"Sharpe Ratio   : {opt_sharpe:.4f}")
else:
    print("Optimization Failed:", result.message)